In [4]:
import pandas as pd


df_closed = pd.DataFrame(columns=['wrist','thumb_tip','index_tip','middle_tip','ring_tip','pinky_tip'])
df_open = pd.DataFrame(columns=['wrist','thumb_tip','index_tip','middle_tip','ring_tip','pinky_tip'])

In [25]:
import tensorflow as tf
import mediapipe as mp
import cv2
import numpy as np
import csv
import math

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# Define landmark indices for wrist and fingertips
wrist_and_finger_tips = [0, 4, 8, 12, 16, 20]

with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            continue

        # Convert BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb.flags.writeable = False

        results = hands.process(frame_rgb)

        frame_rgb.flags.writeable = True
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

        open_label = 1  # for open palm

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks
                mp_drawing.draw_landmarks(frame_bgr, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extract 2D coordinates
                coords_2d = [(lm.x, lm.y) for lm in hand_landmarks.landmark]

                # Get normalization scale: distance between landmark 0 and 12
                p0 = coords_2d[0]
                p12 = coords_2d[12]
                norm_dist = math.dist(p0, p12)

                if norm_dist == 0:
                    continue  # skip bad frame

                # Select only wrist and finger tip coordinates
                selected_points = [coords_2d[i] for i in wrist_and_finger_tips]

                # Normalize and make relative to wrist (point 0)
                wrist = selected_points[0]
                normalized_values = [((x - wrist[0]) / norm_dist, (y - wrist[1]) / norm_dist)
                                     for (x, y) in selected_points]

                distances = [np.sqrt(normalized_values[i][0]**2 + normalized_values[i][1]**2) for i in range(len(normalized_values))]


                df_closed.loc[len(df_closed)] = distances

        else:
            print("No hands detected")

        frame_bgr = cv2.flip(frame_bgr, 1)
        cv2.imshow('frame', frame_bgr)

        if cv2.waitKey(10) == 27:  # ESC key
            break

cap.release()
cv2.destroyAllWindows()


In [16]:

df_open.shape

(5788, 7)

In [15]:
df_open['label'] = 1 # open_label

In [18]:
df_open.to_csv('palm_open.csv',index = False)

In [27]:

df_closed['label'] = 0

In [28]:
df_closed.shape


(5312, 7)

In [30]:
df_closed.to_csv('palm_close.csv',index = False)

In [32]:
df = pd.concat([df_open,df_closed])

In [33]:
df.shape

(11100, 7)

In [35]:
from sklearn.svm import SVC


In [37]:
df.sample(frac=1,random_state = 42)

,wrist,thumb_tip,index_tip,middle_tip,ring_tip,pinky_tip,label
515,0.0,0.732721,0.922441,1.0,0.954563,0.836204,1
1736,0.0,1.393382,1.055731,1.0,0.908371,0.891191,0
1067,0.0,0.886765,0.973209,1.0,0.978824,0.956492,0
3914,0.0,1.269312,1.061261,1.0,0.925815,0.959913,0
3126,0.0,0.644761,0.959993,1.0,0.942137,0.810127,1
...,...,...,...,...,...,...,...
3757,0.0,0.628129,0.954746,1.0,0.945517,0.826438,1
2761,0.0,1.630339,1.151959,1.0,0.895687,0.936185,0
4008,0.0,0.647944,0.948598,1.0,0.963885,0.816461,1
3885,0.0,0.595641,0.951154,1.0,0.938928,0.779994,1


In [43]:
x = df[['thumb_tip','index_tip','ring_tip','pinky_tip']].values
y = df['label'].values

In [44]:
print(x.shape,y.shape)


(11100, 4) (11100,)


In [45]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [47]:
svc_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svc_model.fit(x_train,y_train)
y_pred = svc_model.predict(x_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1041
           1       1.00      1.00      1.00      1179

    accuracy                           1.00      2220
   macro avg       1.00      1.00      1.00      2220
weighted avg       1.00      1.00      1.00      2220



In [1]:
import cv2
import numpy as np
import math
import mediapipe as mp
import pydirectinput
import pickle
import time

with open('svc_model.pkl','rb') as file:

    svc_model = pickle.load(file)

# Initialize Mediapipe Hands and Drawing
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# Define landmark indices for wrist and fingertips
wrist_and_finger_tips = [0, 4, 8, 12, 16, 20]

with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            continue

        # Convert BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb.flags.writeable = False
        results = hands.process(frame_rgb)
        frame_rgb.flags.writeable = True
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        frame = cv2.flip(frame_bgr, 1)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks
                mp_drawing.draw_landmarks(frame_bgr, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extract 2D coordinates
                coords_2d = np.array(
                    [(int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])) for lm in hand_landmarks.landmark],
                    dtype=np.int32
                )

                # Get normalization scale: distance between landmark 0 and 12
                p0 = coords_2d[0]
                p12 = coords_2d[12]
                norm_dist = math.dist(p0, p12)

                if norm_dist == 0:
                    continue  # skip bad frame

                # Select only wrist and finger tip coordinates
                selected_points = [coords_2d[i] for i in wrist_and_finger_tips]

                # Normalize and make relative to wrist (point 0)
                wrist = selected_points[0]
                normalized_values = [
                    ((x - wrist[0]) / norm_dist, (y - wrist[1]) / norm_dist)
                    for (x, y) in selected_points
                ]

                distances = [
                    np.sqrt(x**2 + y**2)
                    for (x, y) in normalized_values
                ]

                # Remove unwanted distances (index 0 = wrist, index 3 = middle finger)
                distances.pop(3)
                distances.pop(0)

                # Predict using pre-trained model (assume svc_model is already loaded)
                pred_prob = svc_model.predict([distances])

                # Get bounding box and draw
                x, y, w, h = cv2.boundingRect(coords_2d)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f'pred_prob: {pred_prob}', (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                if pred_prob == 1:
                    pydirectinput.keyDown('w')
                    time.sleep(1)
                    pydirectinput.keyUp('w')
                elif pred_prob == 0:
                    pydirectinput.keyDown('space')
                    time.sleep(1)
                    pydirectinput.keyUp('space')
        else:
            print("No hands detected")

        cv2.imshow('frame', frame)

        if cv2.waitKey(10) == 27:  # ESC key
            break

cap.release()
cv2.destroyAllWindows()
